In [1]:
import collections
import json
import keras
import numpy as np
import os
import sys
sys.path.append("../../../ecg")
import scipy.stats as sst

import util
import load

Using TensorFlow backend.


In [2]:
model_path = "/deep/group/awni/ecg_models/default/1528249597-44/0.412-0.870-015-0.309-0.892.hdf5"
data_path = "../dev.json"

data = load.load_dataset(data_path)
preproc = util.load(os.path.dirname(model_path))
model = keras.models.load_model(model_path)

100%|██████████| 852/852 [00:00<00:00, 931.25it/s]


In [31]:
data_path = "../train.json"
with open("../train.json", 'r') as fid:
    train_labels = [json.loads(l)['labels'] for l in fid]
counts = collections.Counter(preproc.class_to_int[l[0]] for l in train_labels)
counts = sorted(counts.most_common(), key=lambda x: x[0])
counts = zip(*counts)[1]
smooth = 500
counts = np.array(counts)[None, None, :]
total = np.sum(counts) + counts.shape[1]
prior = (counts + smooth) / float(total)

In [3]:
probs = []
labels = []
for x, y  in zip(*data):
    x, y = preproc.process([x], [y])
    probs.append(model.predict(x))
    labels.append(y)

In [38]:
preds = []
ground_truth = []
for p, g in zip(probs, labels):
    preds.append(sst.mode(np.argmax(p / prior, axis=2).squeeze())[0][0])
    ground_truth.append(sst.mode(np.argmax(g, axis=2).squeeze())[0][0])

In [39]:
import sklearn.metrics as skm
report = skm.classification_report(
            ground_truth, preds,
            target_names=preproc.classes,
            digits=3)
scores = skm.precision_recall_fscore_support(
                    ground_truth,
                    preds,
                    average=None)
print(report)
print "CINC Average {:3f}".format(np.mean(scores[2][:3]))

             precision    recall  f1-score   support

          A      0.859     0.912     0.885        80
          N      0.914     0.923     0.919       508
          O      0.803     0.785     0.794       233
          ~      0.731     0.613     0.667        31

avg / total      0.872     0.873     0.872       852

CINC Average 0.865827
